In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import necessary libraries

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sn
import cv2

import glob

## Define the paths for train, test and validation datasets

In [ ]:
base_path = "../input/balls-image-classification/balls"

train_dir = "../input/balls-image-classification/balls/train"
test_dir = "../input/balls-image-classification/balls/test"
val_dir = "../input/balls-image-classification/balls/valid"

In [ ]:
df = pd.read_csv("../input/balls-image-classification/balls/balls.csv")

# EDA and data visualization

<p style="font-size: 1.5rem">Total number of images</p>

In [ ]:
print(f"Total number of imgaes -- > {len(df)}")

<p style="font-size: 1.5rem">Number of train, test and validation images</p>

In [ ]:
data_count = df['data set'].value_counts()

In [ ]:
print(f"Number of training images --> {data_count[0]}")
print(f"Number of testing images --> {data_count[1]}")
print(f"Number of validation images --> {data_count[2]}")

In [ ]:
plt.figure(figsize=(15, 10))
plt.pie(x=np.array([data_count[0], data_count[1], data_count[2]]), autopct="%.1f%%", explode=[0.2,0, 0], labels=["train", "test", "validation"], pctdistance=0.5)
plt.title("Share of train, test and validation images", fontsize=14);

<p style="font-size: 2rem">Training images comprise 92.3% of the total images</p>

In [ ]:
ball_classes = os.listdir(train_dir)

In [ ]:
len(ball_classes)

<p style="font-size: 2rem"> There are 24 classes of balls present in the dataset</p>

In [ ]:
train_images = glob.glob(f"{train_dir}/*/*.jpg")
test_images = glob.glob(f"{test_dir}/*/*.jpg")
val_images = glob.glob(f"{val_dir}/*/*.jpg")

<p style="font-size: 2rem"> Number of training images available for each class: </p>

In [ ]:
class_dict = {}
for clas in ball_classes:
    num_items = len(os.listdir(os.path.join(train_dir, clas)))
    class_dict[clas] = num_items

In [ ]:
class_dict

In [ ]:
plt.figure(figsize=(15,8))
plt.bar(list(class_dict.keys()), list(class_dict.values()), width=0.4,align="center" )
plt.xticks(rotation=90)

plt.xlabel("Type of balls")
plt.ylabel("Number of balls")
plt.show()

<p style="font-size: 1.5rem"> <em>Paint balls</em> have the least number of images whereas <em>cannon balls</em> the highest </p>

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i,ax in enumerate(axes.flat):
    images = os.listdir(os.path.join(train_dir, ball_classes[i]))
    img = cv2.imread(os.path.join(train_dir, ball_classes[i], images[i]))
    img = cv2.resize(img, (512,512))
    ax.imshow(img)
    ax.set_title(ball_classes[i])
fig.tight_layout()    
plt.show()

# Create image generators

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255.,)
test_datagen = ImageDataGenerator(rescale = 1./255.,)


train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='categorical', target_size = (220, 220))
validation_generator = val_datagen.flow_from_directory(val_dir, batch_size=20, class_mode = 'categorical', target_size=(220, 220))
test_generator = test_datagen.flow_from_directory(test_dir,shuffle=False, batch_size=20, class_mode = 'categorical', target_size=(220, 220))

In [ ]:
input_shape = (220, 220, 3)

# Model creation

In [ ]:
# define the model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=input_shape, include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = 'relu'))
# model.add(Dropout(0.2))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
# model.add(Dropout(0.2))
model.add(Dense(24, activation='softmax'))
model.summary()

# Model Training

In [ ]:
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=["accuracy"])

# callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

history = model.fit(train_generator, validation_data=validation_generator, steps_per_epoch = 100, epochs=30)

# Metrics

In [ ]:
accuracy = history.history['accuracy']
val_accuracy  = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(2, 2, 1)
plt.plot(accuracy, label = "Training accuracy")
plt.plot(val_accuracy, label="Validation accuracy")
plt.legend()
plt.title("Training vs validation accuracy")


plt.subplot(2,2,2)
plt.plot(loss, label = "Training loss")
plt.plot(val_loss, label="Validation loss")
plt.legend()
plt.title("Training vs validation loss")

plt.show()

In [ ]:
pred = model.predict(test_generator)

In [ ]:
pred

In [ ]:
y_pred = np.argmax(pred, axis=1)

In [ ]:
y_pred_class = dict((v,k) for k,v in test_generator.class_indices.items())
    

In [ ]:
y_pred_class

In [ ]:
y_pred

In [ ]:
y_pred = list(map(lambda x: y_pred_class[x], y_pred))

In [ ]:
y_pred

In [ ]:
y_true = test_generator.classes

In [ ]:
y_true = list(map(lambda x: y_pred_class[x], y_true))

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
results = model.evaluate(test_generator)

# Accuracy = 95%

<p style="font-size: 1.5rem"> Please comment, if you have any suggestions.</p>
<p style="font-size: 1.5rem"> Please UPVOTE, if you found this notebook helpful </p>
<p style="font-size: 2rem"> Thank you</p>